In [ ]:


import pandas as pd
import numpy as np
import librosa
# import matplotlib.pylab as plt
# import seaborn as sns
import IPython.display as ipt
import glob
import os
from tinytag import TinyTag

import kagglehub
path = ""
try:
    path = kagglehub.dataset_download("noahbadoa/fma-dataset-100k-music-wav-files")
    print("Path to dataset files:", path)
except Exception as e:
    print("Error:", e)

target_path = path + r'\fma_large'

files = glob.glob(path + r'\**\*.mp3', recursive = True)

extract_features = []

for i in range(20000):
    try: 
        
        tag = TinyTag.get(files[i])
        try:
            y, sr = librosa.load(files[i], sr=22050)
        except Exception as e:
            print(e)
            continue
        
        if y.size == 0 or not np.any(y) or len(y) < 2048:
            print(f"Empty/short audio file skipped")
            continue

        onset_env = librosa.onset.onset_strength(y=y, sr=sr)
        if not np.any(onset_env):
            print("No onset detected, skipped")
            continue

        
        mfcc = librosa.feature.mfcc(y=y, sr=sr)
    
        onset_env = librosa.onset.onset_strength(y=y, sr=sr)
        tempo = librosa.feature.tempo(onset_envelope=onset_env, sr=sr)
    
        centroid = librosa.feature.spectral_centroid(y=y, sr=sr)
    
        spec_rolloff = librosa.feature.spectral_rolloff(y=y, sr=sr)
    
        chroma = librosa.feature.chroma_stft(y=y, sr=sr)
    
        zero_crossing = librosa.feature.zero_crossing_rate(y)
        
        # aggregate matrices 
        mfcc_mean = np.mean(mfcc)
        mfcc_std = np.std(mfcc)
        centroid_mean = np.mean(centroid)
    
        extract_features.append({"mfcc_mean": mfcc_mean,
        "mfcc_std": mfcc_std,
        "tempo": tempo[0],  # tempo is already an array of 1
        "spec_centroid_mean": centroid_mean,
        "spec_rolloff_mean": np.mean(spec_rolloff),
        "chroma_mean": np.mean(chroma),
        "zero_crossing_mean": np.mean(zero_crossing),
        "title": tag.title,
        "artist": tag.artist,
        "genre": tag.genre,
        "duration": tag.duration})
    except Exception as e: 
        print(e)
        continue

feature_df = pd.DataFrame(extract_features)

feature_df.to_csv("features.csv")

print(feature_df)



In [ ]:
import pandas as pd 



df = pd.read_csv(r'D:\songrec\songrec_metadata\fma_metadata\raw_artists.csv')

pd.set_option('display.max_rows', None)


In [11]:
from sklearn.neighbors import NearestNeighbors as nn 
import pandas as pd
import numpy as np 

df = pd.read_csv('features.csv', usecols=range(7))
test = np.array(df.iloc[0]).reshape(1,-1)
data = np.array(df.values.tolist())
knn = nn(n_neighbors=5, algorithm='kd_tree').fit(data)
distance, indeces = knn.kneighbors(test)

print(distance)
# print(indeces)


[[  0.         233.6141249  308.28580887 333.16281293 392.48637993]]
